In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
akas_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.akas.tsv.gz'
akas_df = pd.read_csv(akas_file, sep='\t', compression='gzip')

/var/folders/h7/97hfnxr93g570s_js7p5lr9h0000gn/T/ipykernel_62406/2025018888.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  akas_df = pd.read_csv(akas_file, sep='\t', compression='gzip')


In [4]:
akas_df = akas_df[akas_df['region'] == 'US']
akas_df.replace('\\N', np.nan, inplace=True)

In [5]:
basics_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.basics.tsv.gz'
basics_df = pd.read_csv(basics_file, sep='\t', compression='gzip')

/var/folders/h7/97hfnxr93g570s_js7p5lr9h0000gn/T/ipykernel_62406/4154712586.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv(basics_file, sep='\t', compression='gzip')


In [6]:
basics_df = basics_df[basics_df['tconst'].isin(akas_df['titleId'])]
basics_df.replace('\\N', np.nan, inplace=True)
basics_df = basics_df.dropna(subset=['runtimeMinutes'])
basics_df = basics_df.dropna(subset=['genres'])
basics_df = basics_df[basics_df['titleType'] == 'movie']
basics_df['startYear'] = basics_df['startYear'].astype(float)
basics_df = basics_df[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <= 2021)]
basics_df = basics_df[~basics_df['genres'].str.contains('Documentary', case=False, na=False)]

In [7]:
ratings_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.ratings.tsv.gz'
ratings_df = pd.read_csv(ratings_file, sep='\t', compression='gzip')

In [8]:
ratings_df = ratings_df[ratings_df['tconst'].isin(basics_df['tconst'])]
ratings_df.replace('\\N', np.nan, inplace=True)

In [9]:
print("Filtered AKAs Info:")
print(akas_df.info())

print("Filtered Basics Info:")
print(basics_df.info())

print("Filtered Ratings Info:")
print(ratings_df.info())

Filtered AKAs Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1466999 entries, 5 to 37255016
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1466999 non-null  object
 1   ordering         1466999 non-null  int64 
 2   title            1466999 non-null  object
 3   region           1466999 non-null  object
 4   language         4139 non-null     object
 5   types            983972 non-null   object
 6   attributes       47506 non-null    object
 7   isOriginalTitle  1465657 non-null  object
dtypes: int64(1), object(7)
memory usage: 100.7+ MB
None
Filtered Basics Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82109 entries, 34800 to 10215213
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82109 non-null  object 
 1   titleType       82109 non-null  object 
 2   primaryTitle    82

In [10]:
output_folder = 'Data'
os.makedirs(output_folder, exist_ok=True)

In [11]:
akas_df.to_csv(os.path.join(output_folder, 'filtered_akas.csv.gz'), index=False, compression='gzip')
basics_df.to_csv(os.path.join(output_folder, 'filtered_basics.csv.gz'), index=False, compression='gzip')
ratings_df.to_csv(os.path.join(output_folder, 'filtered_ratings.csv.gz'), index=False, compression='gzip')